I should write stuff here

## Notes to self
- I removed clm, numclaims. Only focusing on claimcst0. Is there a way to train the model on clm, numclaims such that it somehow improves the accuracy???
- Alot of the claimcst0 have no values and only a few have non-zero value, is there something we can do here???

### Imports & Global Variables

In [30]:
import numpy as np
import pandas as pd
import matplotlib as plt
# place imports back here

In [31]:
import os, sys
PATH = %pwd
PATH = PATH.split(os.path.sep)
PATH = os.path.sep.join(PATH[:-2]) #removes last 2 directories, gets path to repository

### Preprocessing

In [32]:
# (1) Read both train/test + validation & preform equal preprocessing on it
df_t = pd.read_csv(os.path.join(PATH, 'dataset', 'InsNova_data_2023_train.csv'))
df_t['is_val'] = 0
df_v = pd.read_csv(os.path.join(PATH, 'dataset', 'InsNova_data_2023_vh.csv'))
df_v['is_val'] = 1
total_df = pd.concat([df_t,df_v], ignore_index=True)
del df_t, df_v


# Note
col_categorical = ['veh_body', 'gender', 'area', 'engine_type', 'veh_color', 
                'marital_status', 'time_of_week_driven', 'time_driven']
col_continuous = ['veh_value', 'exposure', 'credit_score']
col_discrete = ['veh_age', 'agecat', 'max_power', 'driving_history_score',
                'e_bill', 'trm_len', 'high_education_ind']


# (2) Feature Scaling
# Continuous/Discrete scaling

'''
from sklearn.preprocessing import RobustScaler
rs = RobustScaler()
total_df[col_continuous] = rs.fit_transform(total_df[col_continuous])
total_df[col_discrete] = rs.fit_transform(total_df[col_discrete])
'''

# Categorical processing
total_df = pd.get_dummies(data=total_df,columns=col_categorical, dtype=float)
# Replace whitespace with underscore ' ' -> '_'
total_df.columns = total_df.columns.str.replace(' ', '_')

# (3) separate total_df
df_t = total_df[total_df['is_val'] == 0]
df_t = df_t.drop(columns=['is_val','clm','numclaims']) # !removing clm, numclaims
df_v = total_df[total_df['is_val'] == 1]
df_v = df_v.drop(columns='is_val')
del total_df
print(df_t)

### LightGBM Model

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import lightgbm as lgb

In [34]:
'''
X = df_t.drop(columns=['claimcst0'], axis=1)
y = df_t['claimcst0']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html
model = lgb.LGBMRegressor(n_jobs=-1,
                        early_stopping_round=10)

search_space = {
    'boosting_type':['gbdt','dart'],
    'num_leaves': [15, 31, 63, 127, -1],
    'max_depth': [5, 10, 20, -1],
    'learning_rate': [0.1, 0.2, 0.01, 0.02, 0.001],
    'n_estimators': [100, 1000, 5000],
    'objective': ['regression'],
    'reg_alpha': [0.0, 0.1, 0.2],
    'reg_lambda': [0.0, 0.1, 0.2],
    'n_jobs': [-1],
    'importance_type': ['split', 'gain'],
    }
    
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
grid_search = GridSearchCV(
    estimator=model,
    param_grid=search_space,
    cv=3,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    error_score=0,
    verbose=1)

grid_search.fit(X=X_train, 
                y=y_train,
                eval_set=[X_test, y_test],
                )
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
print(best_params, best_model)


search_space = {
    'boosting_type': ['gbdt'],
    'num_leaves': [31, 63, 127],
    'max_depth': [5, 10, 20],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 500, 1000],
    'reg_alpha': [0.0, 0.1, 0.2],
    'reg_lambda': [0.0, 0.1, 0.2],
    'importance_type': ['split', 'gain'],
}
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=search_space,
    n_iter=100,  # Adjust the number of iterations as needed
    cv=3,
    n_jobs=-1,
    scoring='r2',
    error_score=0,
    verbose=0
)
random_search.fit(X=X_train, y=y_train, eval_set=[(X_test, y_test)])
best_params = random_search.best_params_
best_model = random_search.best_estimator_
print(best_params, best_model)
'''

"\nX = df_t.drop(columns=['claimcst0'], axis=1)\ny = df_t['claimcst0']\n\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html\nmodel = lgb.LGBMRegressor(n_jobs=-1,\n                        early_stopping_round=10)\n\nsearch_space = {\n    'boosting_type':['gbdt','dart'],\n    'num_leaves': [15, 31, 63, 127, -1],\n    'max_depth': [5, 10, 20, -1],\n    'learning_rate': [0.1, 0.2, 0.01, 0.02, 0.001],\n    'n_estimators': [100, 1000, 5000],\n    'objective': ['regression'],\n    'reg_alpha': [0.0, 0.1, 0.2],\n    'reg_lambda': [0.0, 0.1, 0.2],\n    'n_jobs': [-1],\n    'importance_type': ['split', 'gain'],\n    }\n    \n# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html\n# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter\ngrid_search = GridSearchCV(\n    estimator=model,\n    param_grid=search_s

In [35]:
'''
y_pred = best_model.predict(X=X_test)

y_test = y_test.values
a,b = 0,0
for i in range(len(y_pred)):
    print(y_test[i], y_pred[i])
'''

'\ny_pred = best_model.predict(X=X_test)\n\ny_test = y_test.values\na,b = 0,0\nfor i in range(len(y_pred)):\n    print(y_test[i], y_pred[i])\n'

In [36]:
X = df_t.drop(columns=['claimcst0'], axis=1)
y = df_t['claimcst0']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
d_train = lgb.Dataset(data=X_train, label=y_train)
d_valid = lgb.Dataset(data=X_test, label=y_test, reference=d_train)

# https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html
# https://lightgbm.readthedocs.io/en/latest/Parameters.html
# TRY! ranking application
lgb_param ={
    'learning_rate': 0.05,
    'boosting_type': 'dart',
    'objective': 'regression',
    'metric': ['rmse'],
    'max_leaves': 50,
    'max_depth': -1,
    'n_jobs': 8,
    #'lambda_l1': 0.1,  
    'lambda_l2': 0.1,
}

#callbacks = [lgb.early_stopping(20)]
model = lgb.train(
    lgb_param,
    d_train,
    num_boost_round=1000000,
    valid_sets=[d_valid],  # Specify the validation datasets
    #callbacks=callbacks,
)

y_pred = model.predict(X_test)

y_test = y_test.values
for i in range(len(y_pred)):
    print(y_pred[i], y_test[i])



[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001968 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1473
[LightGBM] [Info] Number of data points in the train set: 18095, number of used features: 51
[LightGBM] [Info] Start training from score 159.060205
[1]	valid_0's rmse: 1399.66
[2]	valid_0's rmse: 1399.63
[3]	valid_0's rmse: 1399.62
[4]	valid_0's rmse: 1399.6
[5]	valid_0's rmse: 1399.59
[6]	valid_0's rmse: 1399.56
[7]	valid_0's rmse: 1399.56
[8]	valid_0's rmse: 1399.57
[9]	valid_0's rmse: 1399.55
[10]	valid_0's rmse: 1399.54
[11]	valid_0's rmse: 1399.53
[12]	valid_0's rmse: 1399.53
[13]	valid_0's rmse: 1399.52
[14]	valid_0's rmse: 1399.52
[15]	valid_0's rmse: 1399.51
[16]	valid_0's rmse: 1399.51
[17]	valid_0's rmse: 1399.49
[18]	valid_0's rmse: 1399.48
[19]	valid_0's rmse: 1399.48
[20]	valid_0's rmse: 1399.45
[21]	valid_0's rmse

KeyboardInterrupt: 